### Huge Credit to [oberger4711](https://github.com/oberger4711) on GitHub
This notebook is an adaptation from oberger4711's [kitti_for_yolo](https://github.com/oberger4711/kitti_for_yolo) script.

## Imports and Variables

In [ ]:
from PIL import Image
import os
import csv

# !!! Change to the directory where your data is
os.chdir("/media/roovedot/common/Pr.Inv.JavierFernandez/Labeling")

# !!! Setup Parameters
label_dir = "data_object_label_2/training/label_2"  # Ruta al directorio de las etiquetas
image_2_dir = "data_object_image_2/training/image_2"  # Ruta al directorio de las imágenes
use_dont_care = True  # True if you want to keep "DontCare" Labels, False to ignore them


OUT_LABELS_DIR = "yolo_formated_labels" # directory where the results will be stored

KEY_PEDESTRIAN = "Pedestrian"
KEY_CYCLIST = "Cyclist"
KEY_CAR = "Car"
KEY_VAN = "Van"
KEY_MISC = "Misc"
KEY_TRUCK = "Truck"
KEY_PERSON_SITTING = "Person_sitting"
KEY_TRAM = "Tram" # Tranvía
KEY_DONT_CARE = "DontCare"

CLAZZ_NUMBERS = { # Include here whatever labels you want to keep
            KEY_PEDESTRIAN : 0,
            KEY_CYCLIST : 1,
            KEY_CAR : 2,
            KEY_VAN: 3,
            KEY_TRUCK: 4,
            KEY_MISC: 5,
            KEY_DONT_CARE : 6
        }

## Functions

In [ ]:
def getSampleId(path): # gets the path to a file "/kitti/labels/000123.txt"
    basename = os.path.basename(path) # selects the name of the file "000123.txt"
    # os.path.splitext() returns a tuple with the name and the extension "000123", ".txt"
    return os.path.splitext(basename)[0] # returns the file name "000123"

# Returns, for each class, Its assigned number in CLAZZ_NUMBERS
# use_dont_care toggles saving "DontCare" Labels
def resolveClazzNumberOrNone(clazz, use_dont_care):
    if clazz in CLAZZ_NUMBERS:
        if clazz == KEY_DONT_CARE and not use_dont_care:
            return None
        return CLAZZ_NUMBERS[clazz]
    return None
    
# Calculates bounding bbox coordinates and size relative to the image size (Yolo format)
def convertToYoloBBox(bbox, size):
    # This is taken from https://pjreddie.com/media/files/voc_label.py .
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (bbox[0] + bbox[1]) / 2.0
    y = (bbox[2] + bbox[3]) / 2.0
    w = bbox[1] - bbox[0]
    h = bbox[3] - bbox[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

# Loads the whole sample image and returns its size
def readRealImageSize(img_path):
    return Image.open(img_path).size

# This is not exact for all images but most (and it should be faster).
def readFixedImageSize():
    return (1242, 375)

def parseSample(lbl_path, img_path, use_dont_care):
    with open(lbl_path) as csv_file: # Opens the label file
        # assigns a name to each value in the label object
        reader = csv.DictReader(csv_file, fieldnames=["type", "truncated", "occluded", "alpha", "bbox2_left", "bbox2_top", "bbox2_right", "bbox2_bottom", "bbox3_height", "bbox3_width", "bbox3_length", "bbox3_x", "bbox3_y", "bbox3_z", "bbox3_yaw", "score"], delimiter=" ")
        yolo_labels = [] # Initialize list to store the converted labels
        for row in reader: # Each row in the label file (kitti format) represents 1 label
            clazz_number = resolveClazzNumberOrNone(row["type"], use_dont_care) #get label type
            if clazz_number is not None:
                size = readRealImageSize(img_path)
                #size = readFixedImageSize()
                # Image coordinate is in the top left corner.
                bbox = ( # gets data about the bbox
                        float(row["bbox2_left"]),
                        float(row["bbox2_right"]),
                        float(row["bbox2_top"]),
                        float(row["bbox2_bottom"])
                       )
                yolo_bbox = convertToYoloBBox(bbox, size) # converts the bbox to yolo format
                # Yolo expects the labels in the form:
                # <object-class> <x> <y> <width> <height>.
                yolo_label = (clazz_number,) + yolo_bbox
                yolo_labels.append(yolo_label)
    return yolo_labels


### Debug/test functions:

In [ ]:
###-DEBUG-###

classes = ['Pedestrian', 'Cyclist', 'Car', 'Van', 'Truck', 'Misc', 'DontCare']
for i in classes:
    print(resolveClazzNumberOrNone(i, False))

## Main Function

In [ ]:
def main(label_dir, image_2_dir, use_dont_care):
    if not os.path.exists(OUT_LABELS_DIR):
        os.makedirs(OUT_LABELS_DIR)

    print("Transforming Labels into Yolo Format...")
    sample_img_pathes = [] 
    for dir_path, sub_dirs, files in os.walk(label_dir):
        for file_name in files: # iterates over label files
            if file_name.endswith(".txt"): # grab label files

                lbl_path = os.path.join(dir_path, file_name) # gets complete route to the file
                sample_id = getSampleId(lbl_path) # get file name/id

                img_path = os.path.join(image_2_dir, "{}.png".format(sample_id))
                sample_img_pathes.append(img_path) # Stores routes to each image corresponding to the labels

                yolo_labels = parseSample(lbl_path, img_path, use_dont_care) # transforms labels to yolo format

                # writes labels in yolo format onto files in the output folder
                with open(os.path.join(OUT_LABELS_DIR, "{}.txt".format(sample_id)), "w") as yolo_label_file:
                    for lbl in yolo_labels:
                        yolo_label_file.write("{} {} {} {} {}\n".format(*lbl))

    print("Transformation complete. All labels have been saved in YOLO format.")

# Execute main Functions
main(label_dir, image_2_dir, use_dont_care)